In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, f1_score, roc_auc_score
)
import warnings
warnings.filterwarnings('ignore')
import gc
from sklearn.neighbors import KNeighborsClassifier
import time
from tqdm import tqdm
from sklearn.svm import SVC

# Основные моменты

1. Данные не нормализованы, поэтому надо прописать хороший пайплайн, который будет делать предобработку данных.

2. Проверить работу базовых алгоритмов классификации, при этом для каждой модели провести тщательный подбор гиперпараметров, чтобы выявить те алгоритмы и подходы, которые в этой задаче будут давать наилушие значения:
    * SGDCLassifier
    * KNN
    * SVC

В данной файле рассмотрим только базовые модели, ансабли будут рассмотрены позже


Также лучшие рузальтаты будут записаны в таблицу, а лучшие модели сохранены.

# Data

In [2]:
# Загрузка
train_df = pd.read_csv('./data/train_processed.csv')
val_df = pd.read_csv('./data/val_processed.csv')

X_train_raw = train_df.drop(columns=['TARGET'])
y_train = train_df['TARGET'].astype(np.int8)
X_val_raw = val_df.drop(columns=['TARGET'])
y_val = val_df['TARGET'].astype(np.int8)

# Определяем признаки для нормализации
numeric_features = X_train_raw.select_dtypes(include=['int64', 'float64']).columns
features_to_scale = [col for col in numeric_features if X_train_raw[col].nunique() > 2]

# Нормализация
if features_to_scale:
    scaler = StandardScaler()
    
    # Все нормализуемые признаки -> float32
    X_train_scaled = scaler.fit_transform(X_train_raw[features_to_scale].astype(np.float32))
    X_val_scaled = scaler.transform(X_val_raw[features_to_scale].astype(np.float32))
    
    # Создаем DataFrame
    X_train = pd.DataFrame(X_train_scaled.astype(np.float32), columns=features_to_scale)
    X_val = pd.DataFrame(X_val_scaled.astype(np.float32), columns=features_to_scale)
    
    # Добавляем остальные признаки (тоже в float32)
    other_features = [col for col in X_train_raw.columns if col not in features_to_scale]
    for col in other_features:
        X_train[col] = X_train_raw[col].astype(np.float32)
        X_val[col] = X_val_raw[col].astype(np.float32)
else:
    # Все признаки в float32
    X_train = X_train_raw.astype(np.float32)
    X_val = X_val_raw.astype(np.float32)

# Очистка
del X_train_raw, X_val_raw, train_df, val_df

print(f"Готово! Все в float32")
print(f"X_train: {X_train.shape}, память: {X_train.memory_usage().sum()/1024**2:.1f} MB")

Готово! Все в float32
X_train: (184506, 702), память: 494.1 MB


# Logistic Regression

In [8]:
# 6. Создаем базовую модель SGDClassifier (логистическая регрессия через SGD)
base_model = SGDClassifier(
    loss='log_loss',  # Это делает её логистической регрессией
    random_state=42,
    max_iter=1000,
    tol=1e-3,
    n_jobs=-1,
    early_stopping=True,  # Ранняя остановка
    validation_fraction=0.1,  # Часть данных для валидации
    n_iter_no_change=5  # Остановка если нет улучшений 5 итераций
)

# 7. Определение сетки гиперпараметров для RandomizedSearchCV
param_distributions = {
    'alpha': np.logspace(-6, 1, 20),  # Параметр регуляризации (аналог 1/C)
    'penalty': ['l1', 'l2', 'elasticnet'],
    'l1_ratio': [0, 0.15, 0.5, 0.85, 1],  # Для elasticnet
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 0.5],  # Начальная скорость обучения
    'class_weight': [None, 'balanced']
}

# 8. Настройка RandomizedSearchCV
print("\nЗапуск RandomizedSearchCV для SGDClassifier...")
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,
    cv=3,
    scoring='roc_auc',
    n_jobs=2,
    verbose=2,
    random_state=42,
    refit=True
)

random_search.fit(X_train, y_train)

print("\nЛучшие параметры:")
print(random_search.best_params_)
print(f"\nЛучшее значение ROC-AUC на кросс-валидации: {random_search.best_score_:.4f}")

# 9. Извлечение лучшей модели
best_model = random_search.best_estimator_

# 10. Оценка на валидационной выборке
print("\n" + "="*50)
print("ОЦЕНКА НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ")
print("="*50)

y_val_pred = best_model.predict(X_val)
y_val_pred_proba = best_model.predict_proba(X_val)[:, 1]

print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"F1 Macro: {f1_score(y_val, y_val_pred, average='macro'):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_val, y_val_pred_proba):.4f}")

# 11. Сохранение модели и scaler
print("\nСохранение модели и scaler...")
model_artifacts = {
    'model': best_model,
    'scaler': scaler,
    'numeric_features': numeric_features,
    'best_params': random_search.best_params_,
    'cv_score': random_search.best_score_
}

joblib.dump(model_artifacts, './models/best_sgd_logistic_regression.pkl')
print("Модель сохранена!")


Запуск RandomizedSearchCV для SGDClassifier...
Fitting 3 folds for each of 30 candidates, totalling 90 fits

Лучшие параметры:
{'penalty': 'elasticnet', 'learning_rate': 'invscaling', 'l1_ratio': 0.5, 'eta0': 0.01, 'class_weight': 'balanced', 'alpha': np.float64(0.0008858667904100823)}

Лучшее значение ROC-AUC на кросс-валидации: 0.7622

ОЦЕНКА НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     56537
           1       0.17      0.69      0.27      4965

    accuracy                           0.70     61502
   macro avg       0.57      0.69      0.54     61502
weighted avg       0.90      0.70      0.77     61502

Confusion Matrix:
[[39738 16799]
 [ 1556  3409]]
Accuracy: 0.7016
F1 Macro: 0.5416
ROC-AUC: 0.7617

Сохранение модели и scaler...
Модель сохранена!


In [3]:
# 1. В начале ноутбука создаем пустой список
model_results = []

# 2. После КАЖДОЙ обученной модели:
def log_model(model, X_val, y_val, name):
    """Записывает метрики модели в таблицу"""
    
    y_pred = model.predict(X_val)
    
    result = {
        'model': name,
        'accuracy': round(accuracy_score(y_val, y_pred), 4),
        'f1': round(f1_score(y_val, y_pred, average='macro'), 4)
    }
    
    # ROC-AUC если доступен
    if hasattr(model, 'predict_proba'):
        try:
            y_proba = model.predict_proba(X_val)
            if len(model.classes_) == 2:
                result['roc_auc'] = round(roc_auc_score(y_val, y_proba[:, 1]), 4)
        except:
            pass
    
    model_results.append(result)
    
    # Создаем DataFrame и показываем
    results_df = pd.DataFrame(model_results)
    print(f"\nДобавлена модель: {name}")
    print(results_df.to_string(index=False))
    
    return results_df

artifacts = joblib.load('./models/best_sgd_logistic_regression.pkl')

results = log_model(artifacts['model'], X_val, y_val, 'SGDClassifier')
results.to_csv('results.csv', index=False)



Добавлена модель: SGDClassifier
        model  accuracy     f1  roc_auc
SGDClassifier    0.7016 0.5416   0.7617


# KNN

In [1]:
print("="*70)
print("ОБУЧЕНИЕ SUPPORT VECTOR MACHINE (SVM)")
print("="*70)

# 1. Проверка данных
print(f"Размеры данных: X_train: {X_train.shape}, X_val: {X_val.shape}")
print(f"Типы данных: {X_train.dtypes.unique()}")
print(f"Классы: {np.unique(y_train)}")

# 2. Проверяем баланс классов
print(f"\nБаланс классов в train:")
print(pd.Series(y_train).value_counts(normalize=True))
print(f"\nБаланс классов в val:")
print(pd.Series(y_val).value_counts(normalize=True))

# 3. Определяем, нужно ли использовать подвыборку для поиска
# SVM очень медленная на больших данных
use_subsample_for_search = len(X_train) > 30000

if use_subsample_for_search:
    print(f"\n{'!'*60}")
    print(f"ВНИМАНИЕ: SVM может быть очень медленной на больших данных!")
    print(f"Размер тренировочных данных: {len(X_train)} samples")
    print(f"Использую подвыборку для поиска гиперпараметров...")
    print(f"{'!'*60}")
    
    # Берем подвыборку для быстрого поиска
    search_sample_size = min(15000, len(X_train))
    search_indices = np.random.choice(len(X_train), search_sample_size, replace=False)
    X_train_search = X_train.iloc[search_indices]
    y_train_search = y_train.iloc[search_indices]
    
    print(f"Подвыборка для поиска параметров: {X_train_search.shape}")
    print(f"(Случайная выборка из {len(X_train)} -> {search_sample_size} samples)")
else:
    X_train_search = X_train
    y_train_search = y_train
    print(f"\nИспользую все данные для поиска параметров...")

# 4. Создаем базовую модель SVM
# Начинаем с линейного ядра - он быстрее
base_svm = SVC(
    kernel='rbf',           # Начнем с RBF (можно поменять на 'linear' для скорости)
    probability=True,       # Чтобы были predict_proba для ROC-AUC
    random_state=42,
    cache_size=500,         # Увеличиваем кэш для ускорения
    verbose=False,          # Не выводить подробности обучения
    class_weight='balanced' # Учитываем дисбаланс классов
)

# 5. Определение сетки гиперпараметров для RandomizedSearchCV
# SVM имеет 2 основных гиперпараметра: C и gamma
param_distributions = {
    'C': np.logspace(-3, 3, 20),          # Параметр регуляризации
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 1, 10)),  # Для RBF ядра
    'kernel': ['rbf', 'linear', 'poly'],  # Типы ядер
    'degree': [2, 3, 4],                  # Для полиномиального ядра
    'coef0': [0.0, 0.1, 0.5, 1.0],        # Для poly и sigmoid ядер
    'shrinking': [True, False],           # Использовать ли shrinking heuristic
    'tol': [1e-4, 1e-3, 1e-2],            # Допуск для остановки
    'max_iter': [-1, 1000, 2000]          # -1 = без ограничений
}

# 6. Настройка RandomizedSearchCV с подробным выводом
n_iter = 15  # Уменьшаем количество итераций для скорости

print(f"\nЗапуск RandomizedSearchCV для SVM...")
print(f"Количество итераций: {n_iter}")
print(f"Размер данных для поиска: {X_train_search.shape}")
print(f"Количество фолдов: 3")
print(f"Всего будет обучено: {n_iter * 3} моделей")

random_search = RandomizedSearchCV(
    estimator=base_svm,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=3,                     # 3 фолда
    scoring='roc_auc',
    n_jobs=1,                 # SVM плохо параллелится, лучше 1
    verbose=10,               # БОЛЬШОЙ VERBOSE для детального вывода
    random_state=42,
    refit=True,
    error_score='raise',
    return_train_score=True   # Чтобы видеть score на train
)

# 7. Подбор гиперпараметров
print("\n" + "="*60)
print("НАЧАЛО ПОДБОРА ГИПЕРПАРАМЕТРОВ SVM")
print("="*60)
print("Будет выводиться прогресс по каждой комбинации параметров...")
print("Fitting 3 folds for each of 15 candidates, totalling 45 fits")
print("-"*60)

import time
start_time = time.time()

random_search.fit(X_train_search, y_train_search)

search_time = time.time() - start_time
print(f"\nПоиск гиперпараметров завершен за {search_time/60:.1f} минут")

# 8. Вывод результатов поиска
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПОИСКА ГИПЕРПАРАМЕТРОВ")
print("="*60)

if use_subsample_for_search:
    print(f"Поиск выполнен на подвыборке: {X_train_search.shape}")
else:
    print(f"Поиск выполнен на всех данных: {X_train_search.shape}")

print("\nЛучшие параметры:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nЛучший ROC-AUC на кросс-валидации: {random_search.best_score_:.4f}")

# 9. Показываем топ-5 комбинаций параметров
print("\nТоп-5 лучших комбинаций параметров:")
results_df = pd.DataFrame(random_search.cv_results_)
top_results = results_df.sort_values('mean_test_score', ascending=False).head(5)

for i, (_, row) in enumerate(top_results.iterrows(), 1):
    print(f"\n{i}. Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})")
    params = row['params']
    for key in ['C', 'kernel', 'gamma', 'degree']:
        if key in params:
            print(f"   {key}: {params[key]}")

# 10. Если использовали подвыборку, обучаем финальную модель на всех данных
if use_subsample_for_search:
    print("\n" + "="*50)
    print("ОБУЧЕНИЕ ФИНАЛЬНОЙ SVM НА ВСЕХ ДАННЫХ")
    print("="*50)
    
    best_svm_params = random_search.best_params_
    
    # Убираем параметры, которые могут вызвать проблемы
    if 'max_iter' in best_svm_params and best_svm_params['max_iter'] == -1:
        best_svm_params['max_iter'] = 2000  # Ограничиваем итерации
    
    print(f"Обучаем модель с лучшими параметрами на всех {len(X_train)} samples...")
    print(f"Параметры: {best_svm_params}")
    print("Это может занять значительное время (возможно, несколько часов)...")
    print("Прогресс будет выводиться во время обучения...")
    
    # Создаем финальную модель
    final_svm = SVC(**best_svm_params, 
                   probability=True,
                   random_state=42,
                   cache_size=1000,  # Увеличиваем кэш для больших данных
                   verbose=True)     # Включаем verbose для отслеживания прогресса
    
    # Засекаем время
    train_start = time.time()
    final_svm.fit(X_train, y_train)
    train_time = time.time() - train_start
        
    best_model = final_svm
    print(f"\nФинальная SVM модель обучена на всех данных за {train_time/60:.1f} минут!")
else:
    best_model = random_search.best_estimator_

ОБУЧЕНИЕ SUPPORT VECTOR MACHINE (SVM)


NameError: name 'X_train' is not defined

In [6]:
print("\n" + "="*70)
print("ОЦЕНКА KNN НА 10,000 СЛУЧАЙНЫХ SAMPLES ИЗ ВАЛИДАЦИИ")
print("="*70)



# 1. Выбираем 10,000 случайных samples из валидации
sample_size = 10000
print(f"\n1. Выборка {sample_size} случайных samples из {len(X_val)} валидационных...")

# Создаем случайные индексы без замены
sample_indices = np.random.choice(len(X_val), sample_size, replace=False)
X_val_sample = X_val.iloc[sample_indices]
y_val_sample = y_val.iloc[sample_indices]

print(f"   Размер подвыборки: {X_val_sample.shape}")

# 2. Функция для предсказания с прогресс-баром по батчам
def predict_batched_with_progress(model, X, batch_size=500, desc="Predicting"):
    """Предсказание с прогресс-баром по батчам"""
    predictions = []
    n_samples = len(X)
    
    for i in tqdm(range(0, n_samples, batch_size), desc=desc):
        end_idx = min(i + batch_size, n_samples)
        X_batch = X.iloc[i:end_idx]
        pred_batch = model.predict(X_batch)
        predictions.extend(pred_batch)
    
    return np.array(predictions)

def predict_proba_batched_with_progress(model, X, batch_size=500, desc="Predicting probabilities"):
    """Predict_proba с прогресс-баром по батчам"""
    probabilities = []
    n_samples = len(X)
    
    for i in tqdm(range(0, n_samples, batch_size), desc=desc):
        end_idx = min(i + batch_size, n_samples)
        X_batch = X.iloc[i:end_idx]
        proba_batch = model.predict_proba(X_batch)[:, 1]
        probabilities.extend(proba_batch)
    
    return np.array(probabilities)

# 3. Predict на подвыборке
print(f"\n2. Predict на {sample_size} samples...")
start_time = time.time()
y_val_pred_sample = predict_batched_with_progress(
    best_model, X_val_sample, batch_size=500, desc="Predict"
)
predict_time = time.time() - start_time

# 4. Predict_proba на подвыборке
print(f"\n3. Predict_proba на {sample_size} samples...")
start_time_proba = time.time()
y_val_pred_proba_sample = predict_proba_batched_with_progress(
    best_model, X_val_sample, batch_size=500, desc="Predict probabilities"
)
proba_time = time.time() - start_time_proba

# 5. Расчет всех метрик
print(f"\n4. Расчет метрик на {sample_size} samples...")
accuracy = accuracy_score(y_val_sample, y_val_pred_sample)
f1 = f1_score(y_val_sample, y_val_pred_sample, average='macro')
roc_auc = roc_auc_score(y_val_sample, y_val_pred_proba_sample)

print("\n" + "="*60)
print(f"РЕЗУЛЬТАТЫ НА {sample_size} SAMPLES ИЗ ВАЛИДАЦИИ")
print("="*60)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Macro: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

print(f"\nВремя выполнения:")
print(f"  Predict: {predict_time:.1f} секунд ({predict_time/60:.1f} минут)")
print(f"  Predict_proba: {proba_time:.1f} секунд ({proba_time/60:.1f} минут)")
print(f"  Всего: {predict_time + proba_time:.1f} секунд ({(predict_time + proba_time)/60:.1f} минут)")

print(f"\nОценка времени для полной валидации ({len(X_val)} samples):")
print(f"  Predict: ~{predict_time * len(X_val)/sample_size/60:.1f} минут")
print(f"  Predict_proba: ~{proba_time * len(X_val)/sample_size/60:.1f} минут")
print(f"  Всего: ~{(predict_time + proba_time) * len(X_val)/sample_size/60:.1f} минут")

print("\nClassification Report:")
print(classification_report(y_val_sample, y_val_pred_sample))

print("Confusion Matrix:")
print(confusion_matrix(y_val_sample, y_val_pred_sample))

# 6. Сохранение с полной структурой
print("\n" + "="*50)
print("СОХРАНЕНИЕ МОДЕЛИ KNN")
print("="*50)

model_artifacts = {
    'model': best_model,
    'scaler': scaler,
    'numeric_features': numeric_features,
    'best_params': random_search.best_params_,
    'cv_score': random_search.best_score_,
    'val_metrics': {
        'accuracy': accuracy,
        'f1_macro': f1,
        'roc_auc': roc_auc,
        'sample_size': sample_size,
        'total_val_size': len(X_val)
    },
    'timing': {
        'predict_time_seconds': predict_time,
        'proba_time_seconds': proba_time,
        'total_time_seconds': predict_time + proba_time,
        'estimated_full_predict_time_minutes': predict_time * len(X_val)/sample_size/60,
        'estimated_full_proba_time_minutes': proba_time * len(X_val)/sample_size/60
    },
    'training_info': {
        'model_type': 'KNeighborsClassifier',
        'used_subsample_for_search': use_subsample,
        'search_sample_size': X_train_search.shape[0] if use_subsample else X_train.shape[0],
        'final_training_size': X_train.shape[0],
        'feature_count': X_train.shape[1],
        'validation_sample_size': sample_size,
        'full_validation_size': len(X_val),
        'sample_indices': sample_indices,  # сохраняем индексы для воспроизводимости
        'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

joblib.dump(model_artifacts, './models/best_knn_model_10k_eval.pkl')
print("Модель KNN сохранена как './models/best_knn_model_10k_eval.pkl'")

print("\n" + "="*70)
print(f"ОЦЕНКА ЗАВЕРШЕНА НА {sample_size} SAMPLES!")
print("="*70)


ОЦЕНКА KNN НА 10,000 СЛУЧАЙНЫХ SAMPLES ИЗ ВАЛИДАЦИИ

1. Выборка 10000 случайных samples из 61502 валидационных...
   Размер подвыборки: (10000, 702)

2. Predict на 10000 samples...


Predict: 100%|██████████| 20/20 [04:27<00:00, 13.37s/it]



3. Predict_proba на 10000 samples...


Predict probabilities: 100%|██████████| 20/20 [04:26<00:00, 13.34s/it]



4. Расчет метрик на 10000 samples...

РЕЗУЛЬТАТЫ НА 10000 SAMPLES ИЗ ВАЛИДАЦИИ
Accuracy: 0.9161
F1 Macro: 0.4781
ROC-AUC: 0.6518

Время выполнения:
  Predict: 267.3 секунд (4.5 минут)
  Predict_proba: 266.7 секунд (4.4 минут)
  Всего: 534.1 секунд (8.9 минут)

Оценка времени для полной валидации (61502 samples):
  Predict: ~27.4 минут
  Predict_proba: ~27.3 минут
  Всего: ~54.7 минут

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      9161
           1       0.00      0.00      0.00       839

    accuracy                           0.92     10000
   macro avg       0.46      0.50      0.48     10000
weighted avg       0.84      0.92      0.88     10000

Confusion Matrix:
[[9161    0]
 [ 839    0]]

СОХРАНЕНИЕ МОДЕЛИ KNN
Модель KNN сохранена как './models/best_knn_model_10k_eval.pkl'

ОЦЕНКА ЗАВЕРШЕНА НА 10000 SAMPLES!


По полученным метрикам видим, что значения precision и recall для положительного класса равны 0, что говорит и том, что модель все данные называет 0, и так как этих классов боьшинство, то и значения метрик выгляидит неплохим, а на деле модель всё подряд предсказвыает 0.

Данные из этой модели записывать в матрицу резульаттов даже не будем, чтобы значениями метрик не вводило в заблуждение.

Вычисление гиперпарамтеров и метрик по всей выборке затруднительно, так как алгоритм медленный, а данных для него сильно много, из-за чего приходится использвать подвыборки.

# SVC

In [4]:
print("="*70)
print("ОБУЧЕНИЕ SUPPORT VECTOR MACHINE (SVM)")
print("="*70)

# 1. Проверка данных
print(f"Размеры данных: X_train: {X_train.shape}, X_val: {X_val.shape}")
print(f"Типы данных: {X_train.dtypes.unique()}")
print(f"Классы: {np.unique(y_train)}")


# 3. Определяем, нужно ли использовать подвыборку для поиска
# SVM очень медленная на больших данных
use_subsample_for_search = len(X_train) > 30000

if use_subsample_for_search:
    print(f"\n{'!'*60}")
    print(f"ВНИМАНИЕ: SVM может быть очень медленной на больших данных!")
    print(f"Размер тренировочных данных: {len(X_train)} samples")
    print(f"Использую подвыборку для поиска гиперпараметров...")
    print(f"{'!'*60}")
    
    # Берем подвыборку для быстрого поиска
    search_sample_size = min(10000, len(X_train))
    search_indices = np.random.choice(len(X_train), search_sample_size, replace=False)
    X_train_search = X_train.iloc[search_indices]
    y_train_search = y_train.iloc[search_indices]
    
    print(f"Подвыборка для поиска параметров: {X_train_search.shape}")
    print(f"(Случайная выборка из {len(X_train)} -> {search_sample_size} samples)")
else:
    X_train_search = X_train
    y_train_search = y_train
    print(f"\nИспользую все данные для поиска параметров...")

# 4. Создаем базовую модель SVM
# Начинаем с линейного ядра - он быстрее
base_svm = SVC(
    kernel='rbf',           # Начнем с RBF (можно поменять на 'linear' для скорости)
    probability=True,       # Чтобы были predict_proba для ROC-AUC
    random_state=42,
    cache_size=500,         # Увеличиваем кэш для ускорения
    verbose=False,          # Не выводить подробности обучения
    class_weight='balanced', # Учитываем дисбаланс классов
)

# 5. Определение сетки гиперпараметров для RandomizedSearchCV
# SVM имеет 2 основных гиперпараметра: C и gamma
param_distributions = {
    'C': np.logspace(-3, 3, 20),          # Параметр регуляризации
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 1, 10)),  # Для RBF ядра
    'kernel': ['rbf', 'linear', 'poly'],  # Типы ядер
    'degree': [2, 3, 4],                  # Для полиномиального ядра
    'coef0': [0.0, 0.1, 0.5, 1.0],        # Для poly и sigmoid ядер
    'shrinking': [True, False],           # Использовать ли shrinking heuristic
    'tol': [1e-4, 1e-3, 1e-2],            # Допуск для остановки
    'max_iter': [1000, 2000, 5000, 10000]          # -1 = без ограничений
}

# 6. Настройка RandomizedSearchCV с подробным выводом
n_iter = 15  # Уменьшаем количество итераций для скорости

print(f"\nЗапуск RandomizedSearchCV для SVM...")
print(f"Количество итераций: {n_iter}")
print(f"Размер данных для поиска: {X_train_search.shape}")
print(f"Количество фолдов: 3")
print(f"Всего будет обучено: {n_iter * 3} моделей")

random_search = RandomizedSearchCV(
    estimator=base_svm,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=3,                     # 3 фолда
    scoring='roc_auc',
    n_jobs=1,                 # SVM плохо параллелится, лучше 1
    verbose=10,               # БОЛЬШОЙ VERBOSE для детального вывода
    random_state=42,
    refit=True,
    error_score='raise',
    return_train_score=True   # Чтобы видеть score на train
)

# 7. Подбор гиперпараметров
print("\n" + "="*60)
print("НАЧАЛО ПОДБОРА ГИПЕРПАРАМЕТРОВ SVM")
print("="*60)
print("Будет выводиться прогресс по каждой комбинации параметров...")
print("Fitting 3 folds for each of 15 candidates, totalling 45 fits")
print("-"*60)

import time
start_time = time.time()

random_search.fit(X_train_search, y_train_search)

search_time = time.time() - start_time
print(f"\nПоиск гиперпараметров завершен за {search_time/60:.1f} минут")

# 8. Вывод результатов поиска
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПОИСКА ГИПЕРПАРАМЕТРОВ")
print("="*60)

if use_subsample_for_search:
    print(f"Поиск выполнен на подвыборке: {X_train_search.shape}")
else:
    print(f"Поиск выполнен на всех данных: {X_train_search.shape}")

print("\nЛучшие параметры:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nЛучший ROC-AUC на кросс-валидации: {random_search.best_score_:.4f}")

ОБУЧЕНИЕ SUPPORT VECTOR MACHINE (SVM)
Размеры данных: X_train: (184506, 702), X_val: (61502, 702)
Типы данных: [dtype('float32')]
Классы: [0 1]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ВНИМАНИЕ: SVM может быть очень медленной на больших данных!
Размер тренировочных данных: 184506 samples
Использую подвыборку для поиска гиперпараметров...
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Подвыборка для поиска параметров: (10000, 702)
(Случайная выборка из 184506 -> 10000 samples)

Запуск RandomizedSearchCV для SVM...
Количество итераций: 15
Размер данных для поиска: (10000, 702)
Количество фолдов: 3
Всего будет обучено: 45 моделей

НАЧАЛО ПОДБОРА ГИПЕРПАРАМЕТРОВ SVM
Будет выводиться прогресс по каждой комбинации параметров...
Fitting 3 folds for each of 15 candidates, totalling 45 fits
------------------------------------------------------------
Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3; 1/15] START C=2.976351441631316, coef0=1.

In [5]:
print("\n" + "="*50)
print("ОБУЧЕНИЕ SVM НА 50,000 СЛУЧАЙНЫХ SAMPLES")
print("="*50)

best_svm_params = random_search.best_params_

# Убираем параметры, которые могут вызвать проблемы
if 'max_iter' in best_svm_params and best_svm_params['max_iter'] == -1:
    best_svm_params['max_iter'] = 2000  # Ограничиваем итерации

# Берем 50,000 случайных samples из тренировочных данных
sample_size = 50000
sample_indices = np.random.choice(len(X_train), sample_size, replace=False)
X_train_sample = X_train.iloc[sample_indices]
y_train_sample = y_train.iloc[sample_indices]

print(f"Использую {sample_size} случайных samples из {len(X_train)} тренировочных данных")
print(f"Размер подвыборки: {X_train_sample.shape}")
print(f"Параметры модели: {best_svm_params}")
print("Обучение начнется сейчас...")

# Создаем финальную модель
final_svm = SVC(**best_svm_params, 
                probability=True,
                random_state=42,
                cache_size=1000,  # Увеличиваем кэш для больших данных
                verbose=True)     # Включаем verbose для отслеживания прогресса

# Засекаем время
train_start = time.time()
print(f"\nНачало обучения в {time.strftime('%H:%M:%S')}...")
final_svm.fit(X_train_sample, y_train_sample)
train_time = time.time() - train_start

best_model = final_svm
print(f"\nФинальная SVM модель обучена на {sample_size} samples за {train_time/60:.1f} минут!")
print(f"Обучение завершено в {time.strftime('%H:%M:%S')}")

# Сохраняем информацию о том, на каких данных обучались
training_sample_info = {
    'sample_size': sample_size,
    'sample_indices': sample_indices,
    'training_time_minutes': train_time / 60
}


ОБУЧЕНИЕ SVM НА 50,000 СЛУЧАЙНЫХ SAMPLES
Использую 50000 случайных samples из 184506 тренировочных данных
Размер подвыборки: (50000, 702)
Параметры модели: {'tol': 0.0001, 'shrinking': False, 'max_iter': 5000, 'kernel': 'linear', 'gamma': np.float64(0.05994842503189409), 'degree': 2, 'coef0': 0.5, 'C': np.float64(0.00206913808111479)}
Обучение начнется сейчас...

Начало обучения в 20:55:25...
[LibSVM]
Финальная SVM модель обучена на 50000 samples за 14.3 минут!
Обучение завершено в 21:09:43


In [6]:
# 10. Оценка на валидационной выборке
print("\n" + "="*60)
print("ОЦЕНКА SVM НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ")
print("="*60)

# SVM может быть медленной при предсказании, используем батчи
def predict_svm_batched(model, X, batch_size=1000):
    """Предсказание SVM по батчам"""
    predictions = []
    probabilities = []
    n_samples = len(X)
    
    print(f"Предсказание для {n_samples} samples батчами по {batch_size}...")
    
    for i in tqdm(range(0, n_samples, batch_size), desc="Predicting"):
        end_idx = min(i + batch_size, n_samples)
        X_batch = X.iloc[i:end_idx]
        
        # Предсказания
        pred_batch = model.predict(X_batch)
        predictions.extend(pred_batch)
        
        # Вероятности
        if hasattr(model, 'predict_proba'):
            proba_batch = model.predict_proba(X_batch)[:, 1]
            probabilities.extend(proba_batch)
    
    return np.array(predictions), np.array(probabilities) if probabilities else None

# Быстрое предсказание
y_val_pred, y_val_pred_proba = predict_svm_batched(best_model, X_val, batch_size=2000)

print("\nРасчет метрик...")
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

print(f"\nМетрики:")
print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"F1 Macro: {f1_score(y_val, y_val_pred, average='macro'):.4f}")
if y_val_pred_proba is not None:
    print(f"ROC-AUC: {roc_auc_score(y_val, y_val_pred_proba):.4f}")

# 11. Сохранение модели
print("\n" + "="*50)
print("СОХРАНЕНИЕ МОДЕЛИ SVM")
print("="*50)

model_artifacts = {
    'model': best_model,
    'scaler': scaler,
    'numeric_features': numeric_features,
    'best_params': random_search.best_params_,
    'cv_score': random_search.best_score_,
    'val_metrics': {
        'accuracy': accuracy_score(y_val, y_val_pred),
        'f1_macro': f1_score(y_val, y_val_pred, average='macro'),
        'roc_auc': roc_auc_score(y_val, y_val_pred_proba) if y_val_pred_proba is not None else None
    },
    'training_info': {
        'model_type': 'SVC',
        'used_subsample_for_search': use_subsample_for_search,
        'search_sample_size': X_train_search.shape[0] if use_subsample_for_search else X_train.shape[0],
        'final_training_size': X_train.shape[0],
        'feature_count': X_train.shape[1],
        'validation_size': X_val.shape[0],
        'kernel': best_model.kernel,
        'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

joblib.dump(model_artifacts, './models/best_svm_model.pkl')
print("Модель SVM сохранена как './models/best_svm_model.pkl'")

# 12. Дополнительная информация
print("\n" + "="*50)
print("ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ")
print("="*50)

print(f"Размер тренировочных данных: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Ядро SVM: {best_model.kernel}")
print(f"Параметр C: {best_model.C}")
if hasattr(best_model, 'gamma'):
    print(f"Параметр gamma: {best_model.gamma}")
print(f"Использовалась подвыборка для поиска: {'Да' if use_subsample_for_search else 'Нет'}")

if use_subsample_for_search:
    print(f"Размер подвыборки для поиска: {X_train_search.shape[0]} samples")
    print(f"Размер финального обучения: {X_train.shape[0]} samples")

print("\n" + "="*70)
print("Готово! SVM модель обучена и сохранена.")
print("="*70)


ОЦЕНКА SVM НА ВАЛИДАЦИОННОЙ ВЫБОРКЕ
Предсказание для 61502 samples батчами по 2000...


Predicting: 100%|██████████| 31/31 [04:43<00:00,  9.15s/it]


Расчет метрик...
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56537
           1       0.00      0.00      0.00      4965

    accuracy                           0.92     61502
   macro avg       0.46      0.50      0.48     61502
weighted avg       0.85      0.92      0.88     61502

Confusion Matrix:
[[56535     2]
 [ 4965     0]]

Метрики:
Accuracy: 0.9192
F1 Macro: 0.4790
ROC-AUC: 0.6261

СОХРАНЕНИЕ МОДЕЛИ SVM
Модель SVM сохранена как './models/best_svm_model.pkl'

ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ
Размер тренировочных данных: 184506 samples, 702 features
Ядро SVM: linear
Параметр C: 0.00206913808111479
Параметр gamma: 0.05994842503189409
Использовалась подвыборка для поиска: Да
Размер подвыборки для поиска: 10000 samples
Размер финального обучения: 184506 samples

Готово! SVM модель обучена и сохранена.


Опять таки ситуация схожа с KNN, так как все пирмеры были определены как 0 класс, и лишь 2 примера как положительный. Увеличение выборки, на которой искать гиперпарметры, а также увеличение выборки, на которой обучать итогувю модель врятли сильно улучшит ситуацию. Данные модели для нашей задачи мало применимы из-за большого дисбаланса классов. 

Данные по этой модели также не будут вноситься в итогувую таблицу с моделями, так как там будет происхожить сравнение в выбор модели с лучшими результатми.